### Guide: https://www.linkedin.com/pulse/fine-tune-nlp-model-run-sentiment-analysis-2023-updated-vergara/

#### WandB: https://docs.wandb.ai/guides/integrations/huggingface

#### WandB: https://colab.research.google.com/github/wandb/examples/blob/master/colabs/huggingface/Optimize_Hugging_Face_models_with_Weights_%26_Biases.ipynb

In [1]:
!pip install transformers
!pip install mlflow
!pip install evaluate
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import torch
import pandas as pd
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import numpy as np
from imblearn.datasets import make_imbalance
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline, TrainingArguments, Trainer, BertModel
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from datasets import Dataset,load_dataset, load_from_disk, DatasetDict
import evaluate
import os
import io

In [4]:
# Login to W&B
!wandb login

wandb: Currently logged in as: johanneshp (ccs-project). Use `wandb login --relogin` to force relogin


In [5]:
import wandb

In [6]:
wandb.init(project='01_06')

wandb: Currently logged in as: johanneshp (ccs-project). Use `wandb login --relogin` to force relogin


In [7]:
# Load model and tokenizer

tokenizer = RobertaTokenizer.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment-latest')
model = RobertaForSequenceClassification.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment-latest', num_labels=3)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
text_df = pd.read_csv('text.csv', index_col=0)

In [9]:
# Fine-tuning model

dataset = Dataset.from_pandas(text_df,preserve_index=False)

train_test = dataset.train_test_split(shuffle = True, seed = 200, test_size=0.3)
test_val = train_test['test'].train_test_split(shuffle = True, seed = 200, test_size=0.50)
train_val_test_dataset = DatasetDict({
    'train': train_test['train'],
    'test': test_val['test'],
    'dev': test_val['train']})


In [10]:
#Tokenize Dataset
def tokenize_function(examples):
    return tokenizer(examples["comment_text"], padding="max_length", truncation=True,max_length=128)
tokenized_datasets = train_val_test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/801 [00:00<?, ? examples/s]

Map:   0%|          | 0/172 [00:00<?, ? examples/s]

Map:   0%|          | 0/172 [00:00<?, ? examples/s]

In [11]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [12]:
#Load the training arguments and hyperparameters
training_args = TrainingArguments(output_dir="/content/roberta_model",
                                  logging_strategy="epoch",
                                  evaluation_strategy="epoch",
                                  per_device_train_batch_size=16,
                                  per_device_eval_batch_size=16,
                                  num_train_epochs=20,
                                  save_strategy = "epoch",
                                  load_best_model_at_end=True,
                                  learning_rate=1e-6,
                                  report_to="wandb",
                                  # args to prevent disk overload on Google Colab
                                  save_steps = -1,
                                  save_on_each_node = False,
                                  overwrite_output_dir = True
                                  )

In [13]:
#Train the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["dev"],
    compute_metrics=compute_metrics
)
trainer.train()    

wandb.finish()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.730800,0.614100,0.744186
2,0.666600,0.593277,0.738372
3,0.605000,0.583991,0.732558
4,0.576400,0.572362,0.755814
5,0.561600,0.574508,0.750000
6,0.567500,0.575602,0.755814
7,0.585200,0.566568,0.755814
8,0.521100,0.570374,0.779070
9,0.540800,0.568446,0.773256
10,0.495500,0.572771,0.784884


eval/accuracy,▃▂▁▄▃▄▄▇▆█▇▇▆▆▆▆▆▆▆▆
eval/loss,█▅▄▂▂▂▁▂▁▂▂▂▃▄▄▄▄▅▅▅
eval/runtime,▁▂▁▃▂▃▄▄▅▅▄▆▅▆█▆▆▇▇▇
eval/samples_per_second,█▇█▆▇▆▅▅▄▄▄▃▄▃▁▃▂▂▂▂
eval/steps_per_second,█▇█▆▇▆▅▅▄▄▄▃▄▃▁▃▂▂▂▂
train/epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇████
train/global_step,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇████
train/learning_rate,██▇▇▇▆▆▅▅▅▄▄▄▃▃▂▂▂▁▁
train/loss,█▇▅▅▄▄▅▃▄▃▂▂▂▂▂▂▁▁▁▁
train/total_flos,▁
train/train_loss,▁


In [89]:
#saving our text-only model
trainer.save_model("text-only")

In [32]:
predictions = trainer.predict(tokenized_datasets['test'])

In [49]:
#test metrics
predictions[2]

{'test_loss': 0.5446626543998718,
 'test_accuracy': 0.7790697674418605,
 'test_runtime': 1.2912,
 'test_samples_per_second': 133.214,
 'test_steps_per_second': 8.52}

In [66]:
pred_labels = np.array([prediction.argmax() for prediction in predictions[0]])

In [67]:
predictions[1]

array([1, 2, 0, 0, 0, 0, 0, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 2,
       0, 0, 1, 2, 1, 2, 2, 0, 0, 0, 1, 2, 0, 0, 0, 1, 2, 0, 0, 2, 2, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 2,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 2, 2, 0, 2, 0,
       0, 0, 0, 0, 2, 2, 1, 0, 1, 0, 2, 0, 0, 0, 2, 0, 2, 0, 2, 1, 0, 1,
       0, 0, 0, 0, 0, 1, 2, 0, 2, 0, 0, 2, 0, 0, 1, 2, 0, 2, 2, 1, 0, 0,
       1, 0, 2, 1, 0, 1, 0, 0, 0, 0, 0, 2, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1])

In [69]:
pred_labels

array([1, 2, 0, 0, 0, 0, 2, 2, 2, 1, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 2, 2,
       0, 0, 0, 2, 0, 2, 2, 0, 0, 0, 0, 2, 0, 0, 0, 1, 2, 0, 0, 2, 2, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 2,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 2, 0,
       0, 0, 0, 0, 2, 2, 1, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 2, 0, 2,
       0, 0, 2, 0, 0, 1, 2, 0, 1, 0, 0, 2, 2, 0, 2, 2, 0, 2, 2, 2, 0, 0,
       0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0])

In [78]:
test_df = pd.DataFrame(train_val_test_dataset['test'])

In [80]:
#this shows the distribution of annotated labels within each party
test_df.groupby('party')['label'].value_counts()

party          label
conservatives  0        53
               2        15
               1         8
labour         0        37
               1        24
               2        14
libdems        0        14
               1         4
               2         3
Name: label, dtype: int64

In [82]:
test_df['predicted_label'] = pred_labels

In [83]:
#this shows the distribution of predicted labels within each party
test_df.groupby('party')['predicted_label'].value_counts()

party          predicted_label
conservatives  0                  55
               2                  15
               1                   6
labour         0                  51
               2                  17
               1                   7
libdems        0                  13
               2                   6
               1                   2
Name: predicted_label, dtype: int64

In [86]:
#conservatives mean sentiment (just taking the mean of all the predicted labels)
test_df[test_df['party'] == 'conservatives']['predicted_label'].mean()

0.47368421052631576

In [87]:
#labour mean sentiment
test_df[test_df['party'] == 'labour']['predicted_label'].mean()

0.5466666666666666

In [88]:
#libdems mean sentiment
test_df[test_df['party'] == 'libdems']['predicted_label'].mean()

0.6666666666666666